<a href="https://colab.research.google.com/github/mnzk/mycolabo/blob/master/demo_torch_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch

def demo2_01():
    x = torch.randn(4,4)
    y = x.view(16)
    z = x.view(-1, 8)
    print(x.size(),y.size(),z.size())
    display(x)
    display(y)
    display(z[0,0].item())


In [0]:
def demo2_02():
    if not torch.cuda.is_available():
        print("No cuda")
        return
    
    device = torch.device("cuda")
    y = torch.ones_like(x, device=device)
    z = y + x
    display(x)
    display(y)
    display(z)


demo2_02()

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import operator

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)  # 1-input-image, 6-output-channels, 3x3-square-combolution
        self.conv2 = nn.Conv2d(6, 16, 3)
        # affine: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6x6 - image-dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, (2,2))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, n):
        size = n.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


def demo_net():
    net = Net()
    # p = list(net.parameters())
    # print(len(p))
    # display(p[0])

    input = torch.randn(1, 1, 32, 32)
    out = net(input)
    net.zero_grad()
    ret = out.backward(torch.randn(1, 10))



#demo_net()





In [0]:
#http://kaga100man.com/2019/01/05/post-84/

import torch
import torch.nn as nn
import torch.nn.functional as F


class CNN(nn.Module):

    def __init__(self, num_classes):
        super(CNN, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=1),
            nn.BatchNorm2d(16)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=1),
            nn.BatchNorm2d(32)
        )

        self.full_connection = nn.Sequential(
            nn.Linear(in_features=32*26*26, out_features=512),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=512, out_features=num_classes)
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = x.view(x.size(0), 32*26*26)
        y = self.full_connection(x)
        return y
    

def train(loader_train, model_obj, optimizer, loss_fn, device, total_epoch, epoch):

    model_obj.train()  # to train mode

    for data, targets in loader_train:
        data = data.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        output = model_obj(data)
        loss = loss_fn(output, targets)

        loss.backward()
        optimizer.step()

    print ('Epoch [%d/%d], Loss: %.4f' % (epoch, total_epoch, loss.item()))


def test(loader_test, trained_model, device):

    trained_model.eval()  # to eval mode
    correct = 0        # 正解率

    with torch.no_grad():
        for data, targets in loader_test:
            data = data.to(device)
            targets = targets.to(device)

            output = trained_model(data)

            # 推論結果の取得と正誤判定
            _, predicted = torch.max(output.data, 1) # 確率が最大のラベルを取得
            correct += predicted.eq(targets.data.view_as(predicted)).sum() # 正解ならば正解数をカウントアップ
    
    # 正解率を計算
    data_num = len(loader_test.dataset) # テストデータの総数
    print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))


def get_mnist_dataset():
    # 3. MNISTのデータセットを取得
    from sklearn.datasets import fetch_openml
    return fetch_openml('mnist_784', data_home='./')

def main():

    # 1. GPUの設定（PyTorchでは明示的に指定する必要がある）
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"device={device}")

    # 2. ハイパーパラメータの設定（最低限の設定）
    batch_size = 100
    num_classes = 10
    epochs = 3

    # 3. MNISTのデータセットを取得
    mnist = get_mnist_dataset()

    # 4. データの設定（入力データは閉区間[0, 1]に正規化する）
    import numpy as np
    x = mnist.data / 255
    y = np.array([*map(int, mnist.target)])
    # 5. DataLoaderの作成
    from torch.utils.data import TensorDataset, DataLoader
    from sklearn.model_selection import train_test_split

    # 5-1. データを学習用とテスト用に分割
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/7, random_state=0)

    # 5-2. データのフォーマットを変換：PyTorchでの形式 = [画像数，チャネル数，高さ，幅]
    x_train = x_train.reshape(60000, 1, 28, 28)
    x_test = x_test.reshape(10000, 1, 28 ,28)

    # 5-3. PyTorchのテンソルに変換
    x_train = torch.Tensor(x_train)
    x_test = torch.Tensor(x_test)
    y_train = torch.LongTensor(y_train)
    y_test = torch.LongTensor(y_test)

    # 5-4. 入力（x）とラベル（y）を組み合わせて最終的なデータを作成
    ds_train = TensorDataset(x_train, y_train)
    ds_test = TensorDataset(x_test, y_test)

    # 5-5. DataLoaderを作成
    loader_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
    loader_test = DataLoader(ds_test, batch_size=batch_size, shuffle=False)

    # 6. モデル作成
    model = CNN(num_classes=num_classes).to(device)
    print(model) # ネットワークの詳細を確認用に表示

    # 7. 損失関数を定義
    loss_fn = nn.CrossEntropyLoss()

    # 8. 最適化手法を定義（ここでは例としてAdamを選択）
    from torch import optim
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # 9. 学習（エポック終了時点ごとにテスト用データで評価）
    print('Begin train')
    for epoch in range(1, epochs+1):
        train(loader_train, model, optimizer, loss_fn, device, epochs, epoch)
        test(loader_test, model, device)

# %time main()

def test():
    mnist = get_mnist_dataset()
    display(type(mnist))

test()